<a href="https://colab.research.google.com/github/Nicompag/ISYS5002_portfolio/blob/main/ISYS5002_Assignment_20201867_Nico_Engler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Modules

In [ ]:
#!pip install yfinance
#import yfinance as yfin
#yfin.pdr_override()
!pip install kora -q
'''load packages'''
import requests
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd


#1. Get Input

In [ ]:
ticker = input('''Please enter the company's stock market ticker symbol: ''')

# Input validation
#if ticker.isdigit() == True:
#  print('Please enter a valid stock market ticker symbol!')



In [ ]:
def get_input():
  while True:
    ticker = input('''Please enter an Australian stock market ticker symbol: ''')
    if ticker[-3:] == '.ax' or ticker[-3:] == '.AX':
      ticker = ticker
      break
    else:
      print('Please enter an Australian ticker.')
  return ticker

ticker = get_input()
ticker

#2. Get Data

In [ ]:
# Establish Session
url = 'https://au.finance.yahoo.com/'
wd.get(url)

In [ ]:
# Work out URL pattern
template = 'https://au.finance.yahoo.com/quote/{ticker1}/profile?p={ticker2}'

url = template.format(ticker1 = ticker, ticker2 = ticker)
wd.get(url)

In [ ]:
def get_url(ticker):
  template = 'https://au.finance.yahoo.com/quote/{ticker1}/profile?p={ticker2}'
  url = template.format(ticker1 = ticker, ticker2 = ticker)

  return url

In [ ]:
soup = BeautifulSoup(wd.page_source, 'html.parser')

#3. Extract Collection

In [ ]:
table = soup.find_all('tr', {'class': 'C($primaryColor) BdB Bdc($seperatorColor) H(36px)'})
table[4].td.text

In [ ]:
def extract_name(item):
  name = item.td.text
  return name

In [ ]:
item = table[0]
salary = item.find('td', 'Ta(end)')
salary = salary.text

salary[-1]

In [ ]:
def extract_salary(item):
  try:
    salary = item.find('td', 'Ta(end)')
    salary = salary.text 
    if salary[-1] == "M":
      salary = float(salary[:-1])*1000000
    else:
      salary = float(salary[:-1])*1000
  except ValueError:
    salary = 'N/A'
  return salary

In [ ]:
extract_salary(table[6])

In [ ]:
def extract_title(item):
  title = item.find('td', 'Ta(start) W(45%)')
  title = title.text
  return title

In [ ]:
def calculate_tax(income):
  try:
    if income <= 18200:
      tax_payable = 0
    elif 18200 < income <= 45000:
      tax_payable = (income - 18200) * 0.19
    elif 45000 < income <= 120000:
      tax_payable = 5092 + 0.325 * (income - 45000)
    elif 120000 < income <= 180000:
      tax_payable = 29467 + 0.37 * (income - 120000)
    elif income > 180000:
      tax_payable = 51667 + 0.45 * (income - 180000)
  except TypeError:
      tax_payable = 'N/A'
  return tax_payable

In [ ]:
def extract_record(item):
  return {
      'Name': extract_name(item),
      'Title': extract_title(item),
      'Salary': extract_salary(item),
      'Expected income tax': calculate_tax(extract_salary(item))
  }

In [ ]:
for i in table:
  print(extract_name(i))

In [ ]:
records = []
for i in table:
  records.append(extract_record(i))

In [ ]:
df = pd.DataFrame.from_records(records)
df

In [11]:
#store data
place = input('''Where do you want to store the data? 
Press 1 for cvs file,
Press 2 for data,
Press 3 for both.
''')

place

Where do you want to store the data? 
Press 1 for cvs file,
Press 2 for data,
Press 3 for both.
1


'1'

#4. Putting it together

In [12]:
!pip install kora -q
'''load packages'''
import requests
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd

def get_url(ticker):
  template = 'https://au.finance.yahoo.com/quote/{ticker1}/profile?p={ticker2}'
  url = template.format(ticker1 = ticker, ticker2 = ticker)
  return url

def extract_name(item):
  name = item.td.text
  return name

def extract_title(item):
  title = item.find('td', 'Ta(start) W(45%)')
  title = title.text
  return title

def extract_salary(item):
  try:
    salary = item.find('td', 'Ta(end)')
    salary = salary.text
    if salary[-1] == "M":
      salary = float(salary[:-1])*1000000
    else:
      salary = float(salary[:-1])*1000 
  except ValueError:
    salary = 0
  return salary

def calculate_tax(income):
  try:
    if income <= 18200:
      tax_payable = 0
    elif 18200 < income <= 45000:
      tax_payable = (income - 18200) * 0.19
    elif 45000 < income <= 120000:
      tax_payable = 5092 + 0.325 * (income - 45000)
    elif 120000 < income <= 180000:
      tax_payable = 29467 + 0.37 * (income - 120000)
    elif income > 180000:
      tax_payable = 51667 + 0.45 * (income - 180000)
  except TypeError:
      tax_payable = 0
  return tax_payable

def extract_record(item):
  return {
      'Name': extract_name(item),
      'Title': extract_title(item),
      'Salary': extract_salary(item),
      'Expected income tax': calculate_tax(extract_salary(item))
  }

def ticker_validation(dataframe):
  while True:
    if dataframe.empty:
      print('No information under this ticker. Please enter a valid ticker.')
      ticker = get_input()
      url = get_url(ticker)
      wd.get(url)
      soup = BeautifulSoup(wd.page_source, 'html.parser')
      records = []
      table = soup.find_all('tr', {'class': 'C($primaryColor) BdB Bdc($seperatorColor) H(36px)'})
      for i in table:
        records.append(extract_record(i))   
      dataframe = pd.DataFrame.from_records(records)
    else: 
      break
  return dataframe

def get_input():
  while True:
    ticker = input('''Please enter an Australian stock market ticker symbol: ''')
    if ticker[-3:] == '.ax' or ticker[-3:] == '.AX':
      ticker = ticker
      break
    else:
      print('Please enter an Australian ticker.')
  return ticker

In [13]:
ticker = get_input()
url = get_url(ticker)
wd.get(url)
soup = BeautifulSoup(wd.page_source, 'html.parser')
records = []
table = soup.find_all('tr', {'class': 'C($primaryColor) BdB Bdc($seperatorColor) H(36px)'})

for i in table:
  records.append(extract_record(i))

df = pd.DataFrame.from_records(records)
df = ticker_validation(df)
df

Please enter an Australian stock market ticker symbol: nab.ax


,Name,Title,Salary,Expected income tax
0,"Mr. Ross Maxwell McEwan CBE, B.B.S.","Group CEO, MD & Director",2540000.0,1113667.0
1,"Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons)...",Group Chief Financial Officer,1130000.0,479167.0
2,Mr. Patrick F. Wright,Group Exec. of Technology & Enterprise Operations,1720000.0,744667.0
3,Ms. Sharon J. Cook,Group Exec. of Legal & Commercial Services,912980.0,381508.0
4,Ms. Susan Ferrier,Group Exec. of People & Culture,929270.0,388838.5
5,Ms. Angela Mentis B.Bus,"Group Chief Digital, Data & Analytics Officer",1620000.0,699667.0
6,Mr. Shaun Dooley,Chief Risk Officer,1050000.0,443167.0
7,Mr. Andrew Irvine,Chief Exec. of Bus. Private Banking Division,333270.0,120638.5
8,Mr. Lesile D. Matheson,Group Chief Operating Officer,0.0,0.0
9,Ms. Sally Mihell,Head of Investor Relations,0.0,0.0
